In [4]:
!pip install timm

^C


In [1]:
# https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055-2/
# https://github.com/huggingface/pytorch-image-models/tree/main?tab=readme-ov-file
# https://huggingface.co/docs/hub/timm

# import logging
# import os
# import string
# import sys
# import nibabel as nib
# import random
# import numpy as np
# import torch
# import seaborn as sns
# from matplotlib import pyplot as plt
# from sklearn.utils import compute_class_weight
# from torch import nn
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from torch.utils.data import DataLoader, Dataset
# from torch.optim import Adam
# from torchvision.models import resnet18, efficientnet_b0
# from torchvision.transforms import Compose, ToTensor, Normalize, Resize, Pad, RandomHorizontalFlip, RandomVerticalFlip, \
#     RandomRotation, InterpolationMode
# from torch.utils.data import WeightedRandomSampler
# from sklearn.model_selection import KFold
# from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
# 
# ############################################################
# 
# from densenet2d import *
# 
# ############################################################

import timm


C:\Users\48735\Desktop\AGH\AI_ECHO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
timm.list_models('*vit*')

['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'davit_base',
 'davit_base_fl',
 'davit_giant',
 'davit_huge',
 'davit_huge_fl',
 'davit_large',
 'davit_small',
 'davit_tiny',
 'efficientvit_b0',
 'efficientvit_b1',
 'efficientvit_b2',
 'efficientvit_b3',
 'efficientvit_l1',
 'efficientvit_l2',
 'efficientvit_l3',
 'efficientvit_m0',
 'efficientvit_m1',
 'efficientvit_m2',
 'efficientvit_m3',
 'efficientvit_m4',
 'efficientvit_m5',
 'fastvit_ma36',
 'fastvit_mci0',
 'fastvit_mci1',
 'fastvit_mci2',
 'fastvit_s12',
 'fastvit_sa12',
 'fastvit_sa24',
 'fastvit_sa36',
 'fastvit_t8',
 'fastvit_t12',
 'flexivit_base',
 'flexivit_large',
 'flexivit_small',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 

In [2]:

#####################################################
# configs
#####################################################

dataset_dir = "./DATA/"

# class weights must be inversely poropotrional to the class counts
class_weights = {0:4, 1:1, 2:1, 3:5}

# input image sizes
W = 678
H = 850

# Po przerobieniu danych W i H należy zastapić tymi wartościami
# W = 590
# H = 830
PADD = (0, (H-W)//2)

# range for degrees of random rotations
DEGREES = 90


In [3]:
#####################################################
# dataloaders 
#####################################################

train_data, test_data, train_weights, _ = load_dataset(dataset_dir, class_weights)

train_transforms = Compose([
    ToTensor(),
    Pad(PADD),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotation(DEGREES, InterpolationMode.BILINEAR),
    Normalize(mean=[0.5], std=[0.5])
])

test_transforms = Compose([
    ToTensor(),
    Pad(PADD),
    Normalize(mean=[0.5], std=[0.5])
])

trainSampler = WeightedRandomSampler(train_weights, len(train_weights),replacement=True)
train_ds = CustomDataset(data=train_data, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, sampler=trainSampler, num_workers=4)

test_ds = CustomDataset(data=test_data, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle = False, num_workers=4)


In [4]:
for n, (imgs, labels) in enumerate(train_loader):
    print(imgs.shape,labels)
    if n==5:
        break

torch.Size([1, 30, 850, 850]) tensor([3])
torch.Size([1, 30, 850, 850]) tensor([3])
torch.Size([1, 30, 850, 850]) tensor([3])
torch.Size([1, 30, 850, 850]) tensor([2])
torch.Size([1, 30, 850, 850]) tensor([0])
torch.Size([1, 30, 850, 850]) tensor([3])


In [6]:

custom_model = densenet161(input_channels = 30,num_classes=4)

resnet_timm_model = timm.create_model('resnet50d', pretrained=True, in_chans = 30, num_classes = 4)

efficient_timm_model = timm.create_model('efficientnet_b0', pretrained=True, in_chans = 30, num_classes = 4)


In [7]:

# Training time

for imgs, labels in train_loader:
    print(imgs.shape,labels)
    out1 = custom_model(imgs)
    out2 = resnet_timm_model(imgs)
    out3 = efficient_timm_model(imgs)
    print(out1)
    print(out2)
    print(out3)
    CImg(imgs.detach().cpu().numpy()).display();
    break
    

torch.Size([1, 30, 850, 850]) tensor([2])
tensor([[-0.0551,  0.1659, -0.1007, -0.2479]], grad_fn=<AddmmBackward0>)
tensor([[-0.0727,  0.0010, -0.0718, -0.0414]], grad_fn=<AddmmBackward0>)
tensor([[ 1.1143, -0.8841,  1.3331, -2.9833]], grad_fn=<AddmmBackward0>)


: this = 0x6022ce4142d0, size = (850,850,30,1) [82 Mio], data = (float32*)0x7d461a600010..0x7d461f8aefef (non-shared) = [ -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1 -1 ], min = -1, max = 471.961, mean = 31.7059, std = 73.3285, coords_min = (0,0,0,0), coords_max = (341,378,22,0).


In [8]:

# Testing time

custom_model.eval()
resnet_timm_model.eval()
efficient_timm_model.eval()

for imgs, labels in test_loader:
    print(imgs.shape,labels)
    with torch.no_grad():
        out1 = custom_model(imgs)
        out2 = resnet_timm_model(imgs)
        out3 = efficient_timm_model(imgs)
    print(out1)
    print(out2)
    print(out3)
    CImg(imgs.detach().cpu().numpy()).display();
    break


torch.Size([1, 30, 850, 850]) tensor([2])
tensor([[ 0.2434,  0.6755, -0.6558, -0.7655]])
tensor([[-0.0678, -0.0181, -0.0512,  0.1313]])
tensor([[ 9.5248,  2.0851,  5.4184, -6.7913]])


: this = 0x60225b4b8700, size = (850,850,30,1) [82 Mio], data = (float32*)0x6022ba118b80..0x6022bf3c7b5f (non-shared) = [ -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1 -1 ], min = -1, max = 485, mean = 27.7924, std = 65.1237, coords_min = (0,0,0,0), coords_max = (317,621,24,0).
